# 2. Neural Style Transfer on your local machine

To make sure all the scripts that are working correctly, we need to test them on our local machine first. Specifically, we'll be testing the components that will be in our __flask app__ and our __scoring app__.

### Flask App
The __flask app__ is responsible for:
1. preprocessing the images into individual frames and extracting the audio file
2. adding references of the individual images to the Service Bus queue
3. waiting until style transfer has been applied to all images
4. postprocessing the style transfer images by stitching them together into a video and re-adding the audio clip

The __flask app__ contains the following files:
- `preprocess.py` 
- `add_images_to_queue.py` 
- `postprocess.py` 
- `util.py`
- `main.py`

`preprocess.py`, `add_images_to_queue.py`, and `postprocess.py` can be executed independently. This is to allow us to test each piece. The `main.py` file will call functions the aforementioned files. It will start by preprocessing the video then adding images to the service bus queue. Then it will wait for the messages in the queue to be dequeued and processed by another running process. (This occurs in the __scoring app__.) After all the images in the queue are processed, the `main.py` will start to postprocess the output frames, and recreate a new video, but this time with the style transfered frames.

### Scoring App
The __scoring app__ is responsible for:
1. polling the Service Bus queue for new images 
2. applying style transfer to those images and saving them back to storage

The __scoring app__ containers the following files:
- `main.py`
- `process_images_from_queue.py`
- `style_transfer.py`
- `util.py`

`main.py` starts the app by calling on the function in the `process_images_from_queue.py` file. This is responsible for continuously polling the service bus queue and checking for new messages. As images get pulled off the queue, the process will then call the stylize function in the `style_transfer.py` file which performs the actual style transfer. The generated style images are saved back to blob directly from this process. 

---

### Import package and load .env

In [ ]:
from dotenv import set_key, get_key, find_dotenv, load_dotenv
from pathlib import Path
import json
import os

In [ ]:
env_path = find_dotenv(raise_error_if_not_found=True)
load_dotenv(env_path)

### Preprocess video 
Run the preprocessing step on the video `orangutan.mp4` that is in blob. Specify the video (including the extention) and the mounted directory to access blob. 

In [ ]:
!python flask_app/preprocess.py \
    --video orangutan.mp4 \
    --storage-mount-dir data

### Add messages to queue
The preprocessing step splits the video into individual frames. Now add those frames to the queue to be processed later.

In [ ]:
!python flask_app/add_images_to_queue.py \
    --video-name orangutan \
    --storage-mount-dir data

### Run the main style transfer process
The `scoring_app/main.py` file will dequeue images from the queue and apply style transfer to the images.

In [ ]:
!python scoring_app/main.py --terminate

### Postprocess the frames
Now that the frames have been processed, we can stitch then back together and reattach the audio file using the `flask_app/postprocess.py` file.

In [ ]:
!python flask_app/postprocess.pyprocess.pyp/postprocess.py \
    --video-name orangutan \
    --storage-mount-dir data

Check out mounted storage container and make sure that the output video is there. It will be called `{video_name}_processed.mp4`

In [ ]:
!ls -la data/orangutan

### Additional commands to monitor
Use these additional commands to monitor.

```bash
# checkout how many elements are in the queue
!az servicebus queue show \
    --name {get_key(env_path, "SB_QUEUE")} \
    --namespace-name {get_key(env_path, "SB_NAMESPACE")} \
    --resource-group {get_key(env_path, "RESOURCE_GROUP")} \
    --query "countDetails.activeMessageCount"
```

---

Continue to the next [notebook](/notebooks/03_create_aks_cluster.ipynb).